In [ ]:
#
#  Ten-Year_Minimum_Errors.ipynb
#  Student Loans
#
#  Created by Ed Silkworth on 7/29/19.
#  Copyright © 2019 Ed Silkworth. All rights reserved.
#

# for computing ten-year minimum errors
# open in Jupyter Notebook or Jupyter Lab

import multiprocessing as mp ; # for accelerating computations
import time # for timing computations

# given
p_min = 2000 ;
p_max = 10000 ;
N = 40 ;
delta_N = (p_max-p_min)/N ;
APR = 4.45 ;
i = APR/100/12 ; # not compounded
alpha = 1

# test within $0.02 of a_min
test = 2 # see BO() regarding a_min, and see T() for running the test

# check and round quantities to the nearest cent with a function
def CR(x): 
    
    if x*100-floor(x*100) > 0.49999999 and x*100-floor(x*100) < 0.50000001:
        return round(x*100+1)/100
    else: 
        return round(x*100)/100

# compute the monthly principal balance and, consequently, monthly outstanding interest with a function
def BO(i, alpha, p, c): # see apply_async() and T() regarding p and c
    
    # set
    B = [p] ; # meaning B[0] = p
    O = [0] ; # meaning O[0] = 0
    m = 1
    
    # i > 0 and 0 < alpha <= 1
    a_min = ceil( alpha*(p*i)*pow(1+alpha*i,120) / (pow(1+alpha*i,120)-1) *100)/100
    
    # testing minimum
    a_min += 0.01*c

    while B[m-1] - (a_min - CR(alpha*B[m-1]*i)) > 0: # do
        B.append({}) ; O.append({}) ; # otherwise, index m will be out of range
        B[m] = CR(B[m-1] - (a_min - CR(alpha*B[m-1]*i))) ; # check whenever round, and check and round at end
        O[m] = CR(O[m-1] + CR(B[m-1]*i) - CR(alpha*B[m-1]*i)) ;
        m += 1
    
    # set
    n = m
    
    # set
    af = B[n-1] + CR(B[n-1]*i) + O[n-1] ;
    B.append({}) ;
    B[n] = B[n-1] - (af - CR(B[n-1]*i) - O[n-1]) # = 0
    # O[n] = 0

    return B

# for running the test
def T(j, r, loop):
    
    # log potential errors in L
    if len( BO(i, alpha, p_min+delta_N*j, 0) )-1 <= 120: 
        L = [0/100] # meaning potentially no error exists
    else: L = []

    for c in r:
        if len( BO(i, alpha, p_min+delta_N*j, -(c+1)) )-1 <= 120:
            L.append(-(c+1)/100)
        else: None
        if len( BO(i, alpha, p_min+delta_N*j, c+1) )-1 <= 120:
            L.append((c+1)/100)            
        else: None    
    
    # compute any errors, and output them
    if float(min(L)) != 0:
        if float(min(L)) > 0:
            print "%s, +%.2f" %(p_min+delta_N*j, min(L))
        else: 
            print "%s, %.2f" %(p_min+delta_N*j, min(L))
    else: None
    
    # just for ones' information
    if loop == N+1:
        print "\n--\nOut of %s computation(s)" %(N+1)
         
# for implementing multiprocessing
def apply_async():
    
    pool = mp.Pool()
    
    loop = 1 ;
    for j in range(N+1):
        pool.apply_async(T, args=(j, range(test), loop, )) ; # empty slot at end is required
        loop += 1

    pool.close() ;
    pool.join()

print "\nChecking for errors...\n(p, error)\n" # prints first

start = time.time()

if __name__ == "__main__":
    apply_async()                           
else: None

end = time.time()

print "\nCompleted in %.2fs" %(end-start)